In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 载入数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# 输入图像大小 (28 * 28)
# 
# 一个 time_step 输入一行像素 28
n_inputs = 28
# 共 28 个时间步长
max_time = 28

# 隐藏单元
# 实际是 LSTM 输出的结果维度 (100, 1)
# 是将 (28, 1) 映射到 (100, 1)
# 三个控制门的权重大小自动匹配
lstm_size = 100

# 10 分类问题
n_class = 10

# 每个批次样本大小
batch_size = 100
# 计算一共有多少批次
n_batch = mnist.train.num_examples // batch_size

In [4]:
# 初始化两个占位符
# 
# 样本与标签
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)

# 初始化权重和偏置
# 
# LSTM 只初始化一次权重和偏置，是在 LSTM 输出的数据 (100, 1)，进行计算的
weights = tf.Variable(initial_value=tf.truncated_normal(shape=(lstm_size, n_class), stddev=0.1))
biases = tf.Variable(initial_value=tf.constant(value=0.1, shape=(1, n_class)))

In [5]:
# 定义 RNN 网络
def RNN(X, weights, biases):
    '''
    x 为样本
    weight 为权重
    biases 为偏置
    '''
    # 输入数据 inputs=[batch_size, max_time, n_inputs]
    inputs = tf.reshape(tensor=X, shape=(-1, max_time, n_inputs))
    
    # 定义 LSTM 的 cell
    lstm_cell_1 = tf.nn.rnn_cell.BasicLSTMCell(num_units=lstm_size)
    
    # final_state[0] batch_size 的 100
    # final_state[1] cell_final_state 就是 LSTM 输入 (28, 1) 对应的 (100, 1) 的 100
    #
    # output shape
    # If time_major == False (default), this will be a `Tensor` shaped:
    # `[batch_size, max_time, cell.output_size]`.
    # If time_major == True, this will be a `Tensor` shaped:
    # `[max_time, batch_size, cell.output_size]`.
    outputs, final_state = tf.nn.dynamic_rnn(cell=lstm_cell_1, inputs=inputs, dtype=tf.float32)
    
    # 最后 LSTM 的输出
    return tf.nn.softmax(logits=tf.matmul(a=final_state[1], b=weights) + biases)
    

In [6]:
prediction = RNN(x, weights, biases)

# 定义损失函数，使用交叉熵
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))

# 定义优化器，使用 Adam
train = tf.train.AdamOptimizer().minimize(loss=cross_entropy)

# 将结果存放在 bool 列表中
# 
correct_prediction = tf.equal(x=tf.argmax(input=y, axis=1), y=tf.argmax(input=prediction, axis=1))

# 计算准确率
accuracy = tf.reduce_mean(tf.cast(x=correct_prediction, dtype=tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [7]:
# 初始化变量
init_op = tf.global_variables_initializer()
epochs = 20

In [8]:
# 计算图
# 
with tf.Session() as sess:
    sess.run(init_op)
    for epoch in range(epochs):
        for _ in range(n_batch):
            x_data, y_data = mnist.train.next_batch(batch_size=batch_size)
            loss, _ = sess.run(fetches=[cross_entropy, train], feed_dict={x: x_data, y: y_data})
            
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        
#         打印
        print('Epoch=' + str(epoch), '  loss=' + str(loss), ' accuracy=' + str(acc))

Epoch=0   loss=1.57833  accuracy=0.9058
Epoch=1   loss=1.518984  accuracy=0.9419
Epoch=2   loss=1.5127153  accuracy=0.9533
Epoch=3   loss=1.5098492  accuracy=0.9628
Epoch=4   loss=1.499671  accuracy=0.9643
Epoch=5   loss=1.4946195  accuracy=0.968
Epoch=6   loss=1.5146761  accuracy=0.9726
Epoch=7   loss=1.5167738  accuracy=0.9704
Epoch=8   loss=1.4717697  accuracy=0.972
Epoch=9   loss=1.4712433  accuracy=0.9728
Epoch=10   loss=1.4891545  accuracy=0.9791
Epoch=11   loss=1.4711635  accuracy=0.9811
Epoch=12   loss=1.4943824  accuracy=0.9767
Epoch=13   loss=1.4690368  accuracy=0.9812
Epoch=14   loss=1.4757507  accuracy=0.981
Epoch=15   loss=1.4950247  accuracy=0.9785
Epoch=16   loss=1.4620818  accuracy=0.9838
Epoch=17   loss=1.4620258  accuracy=0.9809
Epoch=18   loss=1.5065184  accuracy=0.9778
Epoch=19   loss=1.471343  accuracy=0.983
